In [105]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from importlib import reload 
from art.attacks.evasion import AutoProjectedGradientDescent,ProjectedGradientDescent,AdversarialPatch,BasicIterativeMethod,CarliniL2Method,CarliniLInfMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist
from sklearn.tree import DecisionTreeClassifier
from multi_MPD import UncertaintyTrainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score,recall_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from joblib import dump, load

In [88]:
# Step 0: Define the neural network model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, stride=1)
        self.conv_2 = nn.Conv2d(in_channels=4, out_channels=10, kernel_size=5, stride=1)
        self.fc_1 = nn.Linear(in_features=4 * 4 * 10, out_features=100)
        self.fc_2 = nn.Linear(in_features=100, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 10)
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x

In [89]:
# Step 1: Load the MNIST dataset

(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

# Step 1a: Swap axes to PyTorch's NCHW format

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

In [90]:
# Step 2: Create the model

model = Net()

# Step 2a: Define the loss function and the optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [91]:
# Step 3: Create the ART classifier

classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

In [92]:
# Step 4: Train the ART classifier

classifier.fit(x_train, y_train, batch_size=64, nb_epochs=4)


In [94]:
dump(classifier,'classifier.joblib')

['classifier.joblib']

In [95]:
# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 98.4%


In [96]:
# x_test_hundred=x_test[:100]
# y_test_hundred=y_test[:100]

# Generating AEs

In [129]:
class AEGenerator:
    def __init__(self, attack_name, x_test, y_test):

        self.attack_name = attack_name
        self.x_test = x_test
        self.y_test = y_test

        if attack_name == 'bim':
            self.attack = BasicIterativeMethod(classifier)
        elif attack_name == 'pgdlinf':
            self.attack = ProjectedGradientDescent(estimator=classifier,norm='inf')
        elif attack_name == 'pgdl1':
            self.attack = ProjectedGradientDescent(estimator=classifier,norm=1)
        elif attack_name == 'pgdl2':
            self.attack = ProjectedGradientDescent(estimator=classifier,norm=2,)
        elif attack_name == 'apgdlinf':
            self.attack = AutoProjectedGradientDescent(classifier, norm=np.inf)
        elif attack_name == 'apgdl1':
            self.attack = AutoProjectedGradientDescent(classifier, norm=1)
        elif attack_name == 'apgdl2':
            self.attack = AutoProjectedGradientDescent(classifier, norm=2)
        elif attack_name == 'cwl2':
            self.attack = CarliniL2Method(classifier,max_iter=50)
        elif attack_name == 'cwlinf':
            self.attack = CarliniLInfMethod(classifier,max_iter=50)

        # Generate adversarial examples
        self.x_test_adv = self.attack.generate(self.x_test)

        # Evaluate the ART classifier on the adversarial test examples
        self.adv_preds = classifier.predict(self.x_test_adv)

        # Getting the individual predictions and their indices
        self.ind_pred = np.argmax(self.adv_preds, axis=1) == np.argmax(self.y_test, axis=1)
        self.false_indices = np.where(~self.ind_pred)[0].tolist()

        # Calculating the accuracy
        self.acc_adv = np.sum(np.argmax(self.adv_preds, axis=1) == np.argmax(self.y_test, axis=1)) / len(self.y_test)
        print("Accuracy on {} adversarial test examples: {}%".format(self.attack_name.upper(), self.acc_adv * 100))
        print()
        print(f"The number of {self.attack_name.upper()} samples that successfully evaded the model are - {len(self.false_indices)}")

# BIM ATTACK

In [110]:
 bim_attack = AEGenerator('bim',x_test,y_test)

PGD - Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Accuracy on BIM adversarial test examples: 1.17%

The number of BIM samples that successfully evaded the model are - 9883


In [111]:
dump(bim_attack,'bim_attack.joblib')

['bim_attack.joblib']

# PGD ATTACK

### PGD L INFINITY 

In [112]:
pgdlinf_attack = AEGenerator('pgdlinf',x_test,y_test)

PGD - Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Accuracy on PGDLINF adversarial test examples: 1.17%

The number of PGDLINF samples that successfully evaded the model are - 9883


In [113]:
dump(pgdlinf_attack,'pgdlinf_attack.joblib')

['pgdlinf_attack.joblib']

### PGD L1

In [114]:
pgdl1_attack = AEGenerator('pgdl1',x_test,y_test)

PGD - Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Accuracy on PGDL1 adversarial test examples: 98.41%

The number of PGDL1 samples that successfully evaded the model are - 159


In [115]:
dump(pgdl1_attack,'pgdl1_attack.joblib')

['pgdl1_attack.joblib']

### PGD L2

In [116]:
pgdl2_attack = AEGenerator('pgdl2',x_test,y_test)

PGD - Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Accuracy on PGDL2 adversarial test examples: 92.89%

The number of PGDL2 samples that successfully evaded the model are - 711


In [117]:
dump(pgdl2_attack,'pgdl2_attack.joblib')

['pgdl2_attack.joblib']

# AUTO PGD ATTACK 

### AUTOPGD L INFINITY

In [118]:
apgdlinf_attack = AEGenerator('apgdlinf',x_test,y_test)

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/313 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

...

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Accuracy on APGDLINF adversarial test examples: 1.0%

The number of APGDLINF samples that successfully evaded the model are - 9900


In [119]:
dump(apgdlinf_attack,'apgdlinf_attack.joblib')

['apgdlinf_attack.joblib']

### AUTOPGD L1

In [120]:
apgdl1_attack = AEGenerator('apgdl1',x_test,y_test)

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/313 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

...

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Accuracy on APGDL1 adversarial test examples: 98.4%

The number of APGDL1 samples that successfully evaded the model are - 160


In [121]:
dump(apgdl1_attack,'apgdl1_attack.joblib')

['apgdl1_attack.joblib']

### AUTOPGD L2

In [130]:
apgdl2_attack = AEGenerator('apgdl2',x_test,y_test)

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/313 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

...

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Accuracy on APGDL2 adversarial test examples: 92.60000000000001%

The number of APGDL2 samples that successfully evaded the model are - 740


In [123]:
dump(apgdl2_attack,'apgdl2_attack.joblib')

['apgdl2_attack.joblib']

# CW ATTACK

### CW L2 

In [131]:
cwl2_attack = AEGenerator('cwl2',x_test,y_test)

C&W L_2:   0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy on CWL2 adversarial test examples: 9.93%

The number of CWL2 samples that successfully evaded the model are - 9007


In [125]:
dump(cwl2_attack,'cwl2_attack.joblib')

['cwl2_attack.joblib']

### CW L INFINITY 

In [ ]:
cwlinf_attack = AEGenerator('cwlinf',x_test,y_test)

C&W L_inf:   0%|          | 0/10000 [00:00<?, ?it/s]

In [127]:
dump(cwlinf_attack,'cwlinf_attack.joblib')

['cwlinf_attack.joblib']